In [1]:
%%time 
from data import *
from data.pair_feature import *

CPU times: user 21.7 s, sys: 2.61 s, total: 24.3 s
Wall time: 24.3 s


In [4]:
simple_features_train.shape

(2991396, 9)

In [5]:
simple_features_test.shape

(1044196, 9)

In [6]:
from xgboost.sklearn import XGBClassifier

In [16]:
from sklearn.cross_validation import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(simple_features_train.values, item_pairs_train.isDuplicate.values, test_size=0.2)

In [7]:
model = XGBClassifier()

In [37]:
item_info_train.head()

,categoryID,title,description,images_array,attrsJSON,price,locationID,metroID,lat,lon
itemID,,,,,,,,,,
1,81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"[1064094, 5252822, 6645873, 6960145, 9230265]",{'Вид техники': 'Грузовики'},300000.0,648140,NaN,64.686946,30.815924
3,14,Yamaha r6,Весь в тюнинге.,"[11919573, 14412228, 3204180, 6646877]","{'Вид мотоцикла': 'Спортивные', 'Вид техники':...",300000.0,639040,NaN,55.678037,37.256548
4,84,iPhone 3gs 8gb,"Телефон в хорошем состоянии, трещин и сколов н...","[14384831, 6102021]",{'Вид телефона': 'iPhone'},3500.0,640650,NaN,56.239398,43.460458
7,84,Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской ap...",[],{'Вид телефона': 'Другие марки'},13500.0,662210,NaN,55.777170,37.586194
8,39,Лыжные ботинки,"Лыжные ботинки в хорошем состоянии, 34 размер","[13718854, 4787310]",{'Вид товара': 'Зимние виды спорта'},500.0,624360,NaN,55.777170,37.586194


In [32]:
model.fit(X_train, y_train, eval_metric='auc', eval_set=[(X_test, y_test)])

[0]	validation_0-auc:0.746837
[1]	validation_0-auc:0.755587
[2]	validation_0-auc:0.756284
[3]	validation_0-auc:0.75765
[4]	validation_0-auc:0.758805
[5]	validation_0-auc:0.764109
[6]	validation_0-auc:0.765302
[7]	validation_0-auc:0.768004
[8]	validation_0-auc:0.768
[9]	validation_0-auc:0.768049
[10]	validation_0-auc:0.768709
[11]	validation_0-auc:0.769508
[12]	validation_0-auc:0.77042
[13]	validation_0-auc:0.771376
[14]	validation_0-auc:0.771662
[15]	validation_0-auc:0.771901
[16]	validation_0-auc:0.772379
[17]	validation_0-auc:0.773065
[18]	validation_0-auc:0.773906
[19]	validation_0-auc:0.774738
[20]	validation_0-auc:0.775083
[21]	validation_0-auc:0.775325
[22]	validation_0-auc:0.775734
[23]	validation_0-auc:0.776163
[24]	validation_0-auc:0.776408
[25]	validation_0-auc:0.776774
[26]	validation_0-auc:0.777203
[27]	validation_0-auc:0.777463
[28]	validation_0-auc:0.777736
[29]	validation_0-auc:0.777846
[30]	validation_0-auc:0.777972
[31]	validation_0-auc:0.778158
[32]	validation_0-auc:0

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [35]:
pd.DataFrame({'feature': simple_features_train.columns,
              'importance': model.feature_importances_}).sort('importance', ascending=False)

/home/lq/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,feature,importance
0,attrsJSON_item_jaccard,0.357143
2,price_diff_ratio,0.302857
1,attrsJSON_key_jaccard,0.085714
8,same_title,0.081429
7,same_metroID,0.054286
4,same_description,0.048571
5,same_lat_lon,0.038571
6,same_locationID,0.031429
3,same_categoryID,0.000000


In [38]:
%time model.fit(simple_features_train, item_pairs_train.isDuplicate)

CPU times: user 2min 56s, sys: 2.69 s, total: 2min 58s
Wall time: 52.4 s


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [40]:
model.classes_

array([0, 1])

In [41]:
prob_test = model.predict_proba(simple_features_test)[:,1]

In [43]:
item_pairs_test.head()

,id,itemID_1,itemID_2
0,0,5,4670875
1,1,5,787210
2,2,6,1705280
3,3,11,3020777
4,4,23,5316130


In [45]:
pd.DataFrame({'id': item_pairs_test['id'], 'probability': prob_test}).to_csv('baseline.csv', index=False)